На сайте запущен А/В тест с целью увеличить доход.  
В приложенном excel файле вы найдете сырые данные по результатам эксперимента – user_id, тип выборки variant_name и доход принесенный пользователем revenue.  
Проанализируйте результаты эксперимента и напишите свои рекомендации менеджеру.

In [53]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

In [54]:
df = pd.read_excel('../data/AB_Test_Results(2).xlsx')

/home/anatoliy/.local/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   USER_ID       10000 non-null  int64  
 1   VARIANT_NAME  10000 non-null  object 
 2   REVENUE       10000 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 234.5+ KB


In [56]:
df.describe()

,USER_ID,REVENUE
count,10000.000000,10000.000000
mean,4981.080200,0.099447
std,2890.590115,2.318529
min,2.000000,0.000000
25%,2468.750000,0.000000
50%,4962.000000,0.000000
75%,7511.500000,0.000000
max,10000.000000,196.010000


In [57]:
df.head()

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
4,6174,variant,0.0


In [58]:
# проверяем количество уникальных ID, чтобы понять есть ли разные записи с одинаковыми ID
len(df['USER_ID'].unique())

6324

In [59]:
(len(df) - len(df['USER_ID'].unique())) * 100 / len(df)

36.76

Почти 37% данных являются дубликатами.  
Проверим данные на дубликаты по группам.

In [60]:
a = len(df.loc[df['VARIANT_NAME'] == 'control', 'USER_ID'])
b = len(df.loc[df['VARIANT_NAME'] == 'control', 'USER_ID'].unique())
(a - b) * 100 / a


21.12760834670947

In [61]:
a = len(df.loc[df['VARIANT_NAME'] == 'variant', 'USER_ID'])
b = len(df.loc[df['VARIANT_NAME'] == 'variant', 'USER_ID'].unique())
(a - b) * 100 / a


21.57097288676236

Дубликаты распрелелены одинаково в обеих группах, что указывает на не коректную работу алгоритма сплитования.  
Попробуем выделить и проанализировать дублированные данные.

In [62]:
# выделяем повторяющиеся id
repeated_id = []
for key, value in dict(df['USER_ID'].value_counts()).items():
    if value > 1:
        repeated_id.append(key)
len(repeated_id)


2660

In [63]:
# создаём датафрейм с повторяющимися id
repeated_id_df = df.loc[df['USER_ID'].isin(repeated_id)]
repeated_id_df

,USER_ID,VARIANT_NAME,REVENUE
0,737,variant,0.0
1,2423,control,0.0
2,9411,control,0.0
3,7311,control,0.0
6,2849,control,0.0
...,...,...,...
9992,9303,variant,0.0
9993,2400,variant,0.0
9994,3129,control,0.0
9995,1981,control,0.0


In [64]:
# сортируем по полученый датаафрейм по id
repeated_id_df.sort_values(by=['USER_ID'])#.head(40)

,USER_ID,VARIANT_NAME,REVENUE
2406,2,control,0.00
3479,2,control,0.00
7076,2,control,0.00
7409,3,variant,0.00
4145,3,variant,0.00
...,...,...,...
2998,9996,control,0.00
5915,9996,variant,6.46
8970,10000,variant,0.00
8921,10000,control,0.00


Полученые результаты говорят о том что ошибки в работе алгоритма сплитованя привели к тому что данные собраны не коректно и дальнейшая их обработка не целесообразна так как не позволит сделать объективные выводы. Требуется исправить ошибки в алгоритме спллитования и запустить тест сново. При зтом, для чистоты эксперимента, исключить поподание уже протестированых пользователей в повторный тест.